In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from IPython.display import Image
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-05-19 12:09 | by soge | Using ris library version: 1.4.7 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\High_Bike_Crashes

In [2]:
try: 
    db.params['user']
    db2.params['user']
except:
    db = pysqldb.DbConnect(type='MS', server='dot55sql01', database='forms', user = 'arcgis', password = 'arcgis')
    db2 = pysqldb.DbConnect(type='MS', server='dotdevgissql01', database='SafetyMap2', ldap=True)

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-05-19 12:09:43
Query run in 31000 microseconds
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly


Additional database connection details required:
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to SafetyMap2 on dotdevgissql01 - user: None 
Connection established 2020-05-19 12:09:43
Query run in 19000 microseconds

Additional database connection details required:
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



### Lion SIP 1697


In [3]:
db2.connect()


Additional database connection details required:
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to SafetyMap2 on dotdevgissql01 - user: None 
Connection established 2020-05-19 12:09:45


In [4]:
lion_1697 = db2.dfquery("""


SELECT DISTINCT nodeidfrom, nodeidto, masteridfrom, masteridto
FROM [SafetyMap2].[dbo].[lion]
WHERE nodeidfrom in (48959,48966,48968,49038)
or nodeidto in (48959,48966,48968,49038)


"""
)


Additional database connection details required:
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [5]:
lion_1697.nodeidfrom.unique()

array([  48891,   48902,   48957,   48959,   48964,   48966,   48968,
         49038, 9053023, 9053031], dtype=int64)

In [6]:
lion_1697.masteridfrom.unique()

array([48891, 48902, 48957, 48959, 48964, 48966, 48968, 49024, 48952,
       48955], dtype=int64)

### All Injury Crashes on SIP 1697

In [7]:
sip_1697 = db.dfquery("""


WITH data AS (

SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Pedestrian' THEN 1 END),0) PED  
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Bicyclist' THEN 1 END),0) BI 
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Occupant' THEN 1 END),0) MVO 
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND i.INJ_KILLED = 'Injured'
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND nodeid in (48959,48966,48968,49038)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid
)

,data2 AS (
SELECT CAST(d.Year as varchar) 'YEAR', SUM(PED) PED, SUM(BI) BI, SUM(MVO) MVO
FROM data d
GROUP BY d.YEAR
)



SELECT * FROM ( SELECT data2.*, PED + BI + MVO Total
                FROM data2

                UNION 

                SELECT tot.*, PED + BI + MVO Total
                FROM (SELECT 'Total' as 'Year', 
                      sum(data2.PED) PED,
                      sum(data2.BI) BI,
                      sum(data2.MVO) MVO
                      FROM data2
                    ) tot
               )inj_year
ORDER BY inj_year.year




""")

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [8]:
sip_1697

,YEAR,PED,BI,MVO,Total
0,2017,3,1,10,14
1,2018,5,1,14,20
2,2019,2,0,20,22
3,Total,10,2,44,56


### All Crashes on SIP 1697

In [9]:
sip_1697_tot= db.dfquery("""

SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid, c.INTEGRATION_ID
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND nodeid in (48959,48966,48968,49038)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid, c.INTEGRATION_ID


""")

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [10]:
a_1697=(map(int,list(sip_1697_tot.INTEGRATION_ID)))

### SIP 1697 Excel Output

In [11]:
b_1697 = """225719048
225719048
214419048
214419048
214019048
212719048
211719048
212019048
210319048
209919048
209719048
202019048
200719048
196819048
190919048
178919048
178419048
175019048
174719048
174319048
168819048
167619048
165219048
163119048
158019048
149819048
149819048
139119048
135319048
131419048
130619048
123619048
123119048
120919048
120119048
115919048
113419048
112519048
111119048
108719048
105819048
103619048
103619048
91619048
88519048
88219048
76119048
76119048
76119048
67419048
67419048
65819048
64519048
63319048
60919048
56319048
56219048
55719048
51019048
45219048
45219048
35919048
28819048
24719048
19819048
18019048
16819048
10019048
258618048
256118048
255118048
251618048
244418048
242118048
240418048
240118048
239118048
236118048
235818048
234918048
230318048
230018048
220118048
218318048
204318048
204318048
204318048
203018048
200318048
199818048
196718048
192218048
192318048
194418048
187218048
182418048
181518048
180918048
178818048
178518048
178518048
175718048
175618048
173618048
173218048
171718048
171118048
171018048
167718048
165718048
164918048
159518048
158118048
155118048
154318048
149418048
147118048
146918048
146918048
146918048
144518048
142218048
136318048
135218048
127118048
125818048
123318048
121118048
120618048
114218048
110818048
108818048
108018048
108318048
100718048
94618048
92318048
91218048
89518048
86318048
80918048
76118048
72918048
73418048
72318048
71218048
67318048
65218048
67518048
59718048
51518048
47718048
46618048
46118048
46018048
45618048
44518048
44218048
35918048
33218048
29618048
28518048
24518048
10418048
9118048
7018048
3618048
2818048
1518048
273817048
269517048
268817048
268517048
262717048
257217048
255117048
254117048
253017048
251717048
224417048
220717048
220717048
221617048
215417048
213117048
211517048
204417048
194717048
185717048
184817048
184617048
182617048
180917048
161917048
159617048
154517048
151917048
151317048
146117048
143117048
143117048
137017048
136217048
136217048
133917048
130317048
130117048
128017048
121617048
117317048
110317048
101117048
99917048
97517048
96517048
95617048
88617048
83617048
83017048
75017048
70017048
66817048
66617048
65017048
62217048
62317048
57017048
53417048
50617048
45217048
40017048
40117048
38917048
37017048
30017048
29517048
26617048
25517048
23317048
22917048
19217048
16917048
10817048
2117048
"""

In [12]:
b_1697 =map(int,b_1697.splitlines())

In [13]:
[item for item in set(b_1697) if item not in set(a_1697)]

[]

### Lion SIP 1651

In [14]:
lion_1651 = db2.dfquery("""



SELECT * FROM (SELECT DISTINCT STREET
                   ,NODEIDFROM nodeid
                   ,masteridfrom masterid
               FROM [SafetyMap2].[dbo].[lion]
               WHERE mft in (SELECT mft
                             FROM [SafetyMap2].[dbo].[lion] lion
                             WHERE lion.segmentid in (75132,75142,76571,76701,119305))
               AND mft is not null
               ) nf

               UNION 

SELECT * FROM(SELECT DISTINCT STREET
                   ,NODEIDTO nodeid
                   ,masteridto masterid
              FROM [SafetyMap2].[dbo].[lion]
              WHERE mft in (SELECT mft
                                 FROM [SafetyMap2].[dbo].[lion] lion
                                 WHERE lion.segmentid in (75132,75142,76571,76701,119305))
              AND mft is not null
              ) nt


"""
)


Additional database connection details required:
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [15]:
lion_1651.nodeid.unique()

array([45922, 45923, 46024, 46031, 46816, 46884, 46887, 70242, 46817],
      dtype=int64)

In [16]:
lion_1651.masterid.unique()

array([45922, 45923, 46024, 46031, 46884, 46887], dtype=int64)

### All Injury Crashes on SIP 1651

In [17]:
sip_1651 = db.dfquery("""


WITH data as(


SELECT DISTINCT Year, nodeid, masterid, INTEGRATION_ID, PED, BI, MVO 
FROM(

SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid, c.INTEGRATION_ID
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Pedestrian' THEN 1 END),0) PED  
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Bicyclist' THEN 1 END),0) BI 
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Occupant' THEN 1 END),0) MVO 
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND i.INJ_KILLED = 'Injured'
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND nodeid in (45922, 45923, 46024, 46031, 46816, 46884, 46887, 70242, 46817)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid, c.INTEGRATION_ID
) nodes

UNION
SELECT * FROM (
SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid, c.INTEGRATION_ID
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Pedestrian' THEN 1 END),0) PED  
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Bicyclist' THEN 1 END),0) BI 
,coalesce(sum(CASE WHEN i.PED_NONPED = 'Occupant' THEN 1 END),0) MVO 
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND i.INJ_KILLED = 'Injured'
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND masterid in (45922, 45923, 46024, 46031, 46884, 46887)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid, c.INTEGRATION_ID
) masters
)



,data2 AS (
SELECT CAST(d.Year as varchar) 'YEAR', SUM(PED) PED, SUM(BI) BI, SUM(MVO) MVO
FROM data d
GROUP BY d.YEAR
)



SELECT * FROM ( SELECT data2.*, PED + BI + MVO Total
                FROM data2

                UNION 

                SELECT tot.*, PED + BI + MVO Total
                FROM (SELECT 'Total' as 'Year', 
                      sum(data2.PED) PED,
                      sum(data2.BI) BI,
                      sum(data2.MVO) MVO
                      FROM data2
                    ) tot
               )inj_year
ORDER BY inj_year.year



""")

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [18]:
sip_1651

,YEAR,PED,BI,MVO,Total
0,2017,3,0,2,5
1,2018,6,1,0,7
2,2019,4,0,2,6
3,Total,13,1,4,18


### All crashes on SIP 1651

In [19]:
sip_1651_tot = db.dfquery("""



WITH data as(


SELECT DISTINCT Year, nodeid, masterid, INTEGRATION_ID
FROM(

SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid, c.INTEGRATION_ID
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND nodeid in (45922, 45923, 46024, 46031, 46816, 46884, 46887, 70242, 46817)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid, c.INTEGRATION_ID
) nodes

UNION

SELECT * FROM (
SELECT DATEPART(YEAR,c.ACCIDENT_DT) 'Year', nodeid, masterid, c.INTEGRATION_ID
FROM [FORMS].[dbo].[WC_ACCIDENT_F] as c
JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
ON c.INTEGRATION_ID=i.ACCIDENT_ID
WHERE year(c.ACCIDENT_DT) between 2017 and 2019
AND coalesce(c.VOID_STATUS_CD , 'N') = 'N'
AND coalesce(nonmv, 0) = 0
AND masterid in (45922, 45923, 46024, 46031, 46884, 46887)
GROUP BY DATEPART(YEAR,c.ACCIDENT_DT),nodeid, masterid, c.INTEGRATION_ID
) masters
)

SELECT * FROM data


""")

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [20]:
a_1651=(map(int,list(sip_1651_tot.INTEGRATION_ID)))

### SIP 1651 Excel Output

In [21]:
b_1651 = """354119110
328219110
304919110
282319110
282119110
274119110
261819110
142619110
128219110
121019110
121119110
90819110
45619110
5719110
370018110
352818110
338718110
333418110
327418110
282918110
282018110
262718110
166518110
21218110
338217110
323217110
323217110
303017110
281117110
280817110
270017110
210817110
153317110
149517110
114017110
56217110
50317110
44617110
"""

In [23]:
b_1651 =map(int,b_1651.splitlines())

In [24]:
[item for item in set(b_1651) if item not in set(a_1651)]

[]